In [0]:
%spark.conf

spark.executor.instances=2


.
.
.
Нужно скопировать себе эту тетрадку и предоставить доступ к копии на чтение, запись и запуск тетрадки пользователю admin. Параграфы с генерацией данных и созданием семплов запускать не нужно, они оставлены для ознакомления

In [2]:
print("https://arena-hadoop.inno.tech:18088/proxy/" + sc.applicationId)

In [3]:
spark.sql("create database hw_4")

In [4]:
import org.apache.spark.mllib.random.RandomRDDs._
import java.time.LocalDate
import java.time.format.DateTimeFormatter

val dates = (0 to 14).map(LocalDate.of(2020, 11, 1).plusDays(_).format(DateTimeFormatter.ofPattern("yyyy-MM-dd"))).toSeq

def generateCity(r: Double): String = if (r < 0.9) "BIG_CITY" else "SMALL_CITY_" + scala.math.round((r - 0.9) * 1000)

def generateCityUdf = udf(generateCity _)

// spark.sql("drop table hw2.events_full")

for(i <- dates) {
    uniformRDD(sc, 10000000L, 1)
    .toDF("uid")
    .withColumn("date", lit(i))
    .withColumn("city", generateCityUdf($"uid"))
    .selectExpr("date", " sha2(cast(uid as STRING), 256) event_id", "city")
    .withColumn("skew_key", when($"city" === "BIG_CITY", lit("big_event")).otherwise($"event_id"))
    .write.mode("append")
    .partitionBy("date")
    .saveAsTable("hw_4.events_full")
}


In [5]:
spark.table("hw_4.events_full")
.select("event_id")
.sample(0.001)
.repartition(2)
.write.mode("overwrite")
.saveAsTable("hw_4.sample")


In [6]:
spark.table("hw_4.sample")
.limit(100)
.coalesce(1)
.write.mode("overwrite")
.saveAsTable("hw_4.sample_small")

In [7]:
spark.table("hw_4.events_full")
.select("event_id")
.sample(0.003)
.repartition(1)
.write.mode("overwrite")
.saveAsTable("hw_4.sample_big")

In [8]:
spark.table("hw_4.events_full")
.select("event_id")
.sample(0.015)
.repartition(1)
.write.mode("overwrite")
.saveAsTable("hw_4.sample_very_big")

Для упражнений сгрененирован большой набор синтетических данных в таблице hw2.events_full. Из этого набора данных созданы маленькие (относительно исходного набора) таблицы разного размера kotelnikov.sample_[small, big, very_big]. 

Ответить на вопросы:
 * какова структура таблиц
 * сколько в них записей 
 * сколько места занимают данные
 

In [11]:
sample_very_big_df = spark.table("hw_4.sample_very_big")
sample_big_df = spark.table("hw_4.sample_big")
sample_small_df = spark.table("hw_4.sample_small")

In [12]:
sample_very_big_df.count()

In [13]:
sample_big_df.count()

In [14]:
sample_small_df.count()

.
.
.

Получить планы запросов для джойна большой таблицы hw2.events_full с каждой из таблиц hw2.sample, hw2.sample_big, hw2.sample_very_big по полю event_id. В каких случаях используется BroadcastHashJoin? 

BroadcastHashJoin автоматически выполняется для джойна с таблицами, размером меньше параметра spark.sql.autoBroadcastJoinThreshold. Узнать его значение можно командой spark.conf.get("spark.sql.autoBroadcastJoinThreshold").

In [16]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

In [17]:
#BroadcastHashJoin т.к. размер таблицы sample меньше параметра spark.sql.autoBroadcastJoinThreshold
spark.table("hw_4.events_full").join(spark.table("hw_4.sample"), "event_id", "leftsemi").explain()

In [18]:
#SoftMergeJoin
spark.table("hw_4.events_full").join(sample_big_df, "event_id", "leftsemi").explain()

In [19]:
#SoftMergeJoin
spark.table("hw_4.events_full").join(sample_very_big_df, "event_id", "leftsemi").explain()

.
.
.

Выполнить джойны с таблицами  hw2.sample,  hw2.sample_big в отдельных параграфах, чтобы узнать время выполнения запросов (например, вызвать .count() для результатов запросов). Время выполнения параграфа считается автоматически и указывается в нижней части по завершении

Зайти в spark ui (ссылку сгенерировать в следующем папраграфе). Сколько tasks создано на каждую операцию? Почему именно столько? Каков DAG вычислений?  

In [21]:
spark.table("hw_4.events_full").join(spark.table("hw_4.sample"), "event_id", "leftsemi").count()

In [22]:
#количество tasks 82
#DAG 2 этапа

In [23]:
spark.table("hw_4.events_full").join(sample_big_df, "event_id", "leftsemi").count()

In [24]:
#количество tasks 284
#DAG: 4 этапа

In [25]:
println("arena-hadoop.inno.tech:18088/proxy/" + sc.applicationId + "/jobs/")

Оптимизировать джойн с таблицами hw2.sample_big, hw2.sample_very_big с помощью broadcast(df). Выполнить запрос, посмотреть в UI, как поменялся план запроса, DAG, количество тасков. Второй запрос не выполнится 

In [27]:
from pyspark.sql.functions import broadcast

In [28]:
spark.table("hw_4.events_full")\
.join(broadcast(spark.table("hw_4.sample_big")), "event_id").count()

In [29]:
#DAG стал в 2 этапа, количество тасков 82

In [30]:
spark.table("hw_4.events_full")\
.join(broadcast(spark.table("hw_4.sample_big")), "event_id").explain()

In [31]:
spark.table("hw_4.events_full")\
.join(broadcast(spark.table("hw_4.sample_very_big")), "event_id").count()

.
.
.

Отключить автоматический броадкаст командой spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1"). Сделать джойн с семплом hw_4.sample, сравнить время выполнения запроса.


In [33]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [34]:
spark.table("hw_4.events_full")\
.join(spark.table("hw_4.sample"), "event_id").count()

In [35]:
#57 c vs  6 мин 34с

In [38]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "26214400")

In [39]:
spark.sql("clear cache")

.
.
.

В процессе обработки данных может возникнуть перекос объёма партиций по количеству данных (data skew). В таком случае время выполнения запроса может существенно увеличиться, так как данные распределятся по исполнителям неравномерно. В следующем параграфе происходит инициализация датафрейма, этот параграф нужно выполнить, изменять код нельзя. В задании нужно работать с инициализированным датафреймом.

Датафрейм разделен на 30 партиций по ключу city, который имеет сильно  неравномерное распределение.

In [41]:
 
from pyspark.sql.functions import col

skew_df = spark.table("hw_4.events_full")\
.where("date = '2020-11-01'")\
.repartition(30, col("city"))\
.cache()

skew_df.count()

In [42]:
sc.applicationId

.
.
.

Посчитать количество event_count различных событий event_id , содержащихся в skew_df с группировкой по городам. Результат упорядочить по event_count.

В spark ui в разделе jobs выбрать последнюю, в ней зайти в stage, состоящую из 30 тасков (из такого количества партиций состоит skew_df). На странице стейджа нажать кнопку Event Timeline и увидеть время выполнения тасков по экзекьюторам. Одному из них выпала партиция с существенно большим количеством данных. Остальные экзекьюторы в это время бездействуют -- это и является проблемой, которую предлагается решить далее.

In [44]:
skew_df.show()

In [45]:
import pyspark.sql.functions as f

skew_df.groupby('city')\
.agg(f.count('event_id').alias('event_count'))\
.orderBy(f.col('event_count').desc())\
.show()

.
.
.

один из способов решения проблемы агрегации по неравномерно распределенному ключу является предварительное перемешивание данных. Его можно сделать с помощью метода repartition(p_num), где p_num -- количество партиций, на которые будет перемешан исходный датафрейм

In [47]:
def print_parts(df):
    ret = df.rdd.mapPartitions(lambda x: [len(list(x))]).collect()
    print(ret)

In [48]:
balanced_df = skew_df.repartition(100)
print_parts(balanced_df)

.
.
.
Другой способ исправить неравномерность по ключу -- создание синтетического ключа с равномерным распределением. В нашем случае неравномерность исходит от единственного значения city='BIG_CITY', которое часто повторяется в данных и при группировке попадает к одному экзекьютору. В таком случае лучше провести группировку в два этапа по синтетическому ключу CITY_SALT, который принимает значение BIG_CITY_rand (rand -- случайное целое число) для популярного значения BIG_CITY и CITY для остальных значений. На втором этапе восстанавливаем значения CITY и проводим повторную агрегацию, которая не занимает времени, потому что проводится по существенно меньшего размера данным. 

Такая же техника применима и к джойнам по неравномерному ключу, см, например https://itnext.io/handling-data-skew-in-apache-spark-9f56343e58e8

Что нужно реализовать:
* добавить синтетический ключ
* группировка по синтетическому ключу
* восстановление исходного значения
* группировка по исходной колонке

In [50]:
import pyspark.sql.functions as f
from pyspark.sql.functions import expr

BIG_CITY_rand = expr("""pmod(round(rand() * 100, 0), 100)""").cast("integer")
CITY_SALT = f.when(f.col('city')=='BIG_CITY', BIG_CITY_rand).otherwise('CITY')

salted = skew_df.withColumn("salt", CITY_SALT)

salted.groupBy('city','salt')\
.agg(f.count('event_id').alias('event_count'))\
.orderBy(f.col('event_count').desc()) \
.show()


In [51]:
salted.groupBy('city','salt')\
.agg(f.collect_list(col("event_id")).alias("ids"))\
.groupBy('city')\
.agg(f.collect_list(col("ids")).alias("ids"))\
.show(20,100)

In [52]:
spark.stop